# Kuzu

>[Kùzu](https://kuzudb.com) is an embeddable property graph database management system built for query speed and scalability.
> 
> Kùzu has a permissive (MIT) open source license and implements [Cypher](https://en.wikipedia.org/wiki/Cypher_(query_language)), a declarative graph query language that allows for expressive and efficient data querying in a property graph.
> It uses columnar storage and its query processor contains novel join algorithms that allow it to scale to very large graphs without sacrificing query performance.
> 
> This notebook shows how to use LLMs to provide a natural language interface to [Kùzu](https://kuzudb.com) database with Cypher.

## Setting up

Kùzu is an embedded database (it runs in-process), so there are no servers to manage.
Simply install it via its Python package:

```bash
pip install kuzu
```

Create a database on the local machine and connect to it:

In [4]:
import kuzu

db = kuzu.Database("test_db")
conn = kuzu.Connection(db)

First, we create the schema for a simple movie database:

In [5]:
conn.execute("CREATE NODE TABLE Movie (name STRING, PRIMARY KEY(name))")
conn.execute(
    "CREATE NODE TABLE Person (name STRING, birthDate STRING, PRIMARY KEY(name))"
)
conn.execute("CREATE REL TABLE ActedIn (FROM Person TO Movie)")

Then we can insert some data.

In [6]:
conn.execute("CREATE (:Person {name: 'Al Pacino', birthDate: '1940-04-25'})")
conn.execute("CREATE (:Person {name: 'Robert De Niro', birthDate: '1943-08-17'})")
conn.execute("CREATE (:Movie {name: 'The Godfather'})")
conn.execute("CREATE (:Movie {name: 'The Godfather: Part II'})")
conn.execute(
    "CREATE (:Movie {name: 'The Godfather Coda: The Death of Michael Corleone'})"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Al Pacino' AND m.name = 'The Godfather Coda: The Death of Michael Corleone' CREATE (p)-[:ActedIn]->(m)"
)
conn.execute(
    "MATCH (p:Person), (m:Movie) WHERE p.name = 'Robert De Niro' AND m.name = 'The Godfather: Part II' CREATE (p)-[:ActedIn]->(m)"
)

## Creating `KuzuQAChain`

We can now create the `KuzuGraph` and `KuzuQAChain`. To create the `KuzuGraph` we simply need to pass the database object to the `KuzuGraph` constructor.

In [7]:
from langchain_openai import ChatOpenAI
from langchain.chains import KuzuQAChain
from langchain_community.graphs import KuzuGraph

In [8]:
graph = KuzuGraph(db)

In [31]:

chain = KuzuQAChain.from_llm(
    llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k", api_key="API_KEY"),
    graph=graph,
    verbose=True,
)


## Refresh graph schema information

If the schema of database changes, you can refresh the schema information needed to generate Cypher statements.
You can also display the schema of the Kùzu graph as demonstrated below.

In [ ]:
# graph.refresh_schema()

In [26]:
print(graph.get_schema)

Node properties: [{'properties': [('name', 'STRING')], 'label': 'Movie'}, {'properties': [('name', 'STRING'), ('birthDate', 'STRING')], 'label': 'Person'}]
Relationships properties: [{'properties': [], 'label': 'ActedIn'}]
Relationships: ['(:Person)-[:ActedIn]->(:Movie)']



## Querying the graph

We can now use the `KuzuQAChain` to ask question of the graph

In [32]:
chain.invoke("Who acted in The Godfather: Part II?")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ActedIn]->(m:Movie)
WHERE m.name = 'The Godfather: Part II'
RETURN p.name
Full Context:
[{'p.name': 'Al Pacino'}, {'p.name': 'Robert De Niro'}]

> Finished chain.


'Al Pacino, Robert De Niro acted in The Godfather: Part II.'

In [33]:
chain.invoke("Robert De Niro played in which movies?")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ActedIn]->(m:Movie)
WHERE p.name = 'Robert De Niro'
RETURN m.name
Full Context:
[{'m.name': 'The Godfather: Part II'}]

> Finished chain.


{'query': 'Robert De Niro played in which movies?',
 'result': 'Robert De Niro played in The Godfather: Part II.'}

In [34]:
chain.invoke("How many actors played in the Godfather: Part II?")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (:Person)-[:ActedIn]->(:Movie {name: 'Godfather: Part II'})
RETURN count(*)
Full Context:
[{'COUNT_STAR()': 0}]

> Finished chain.


{'query': 'How many actors played in the Godfather: Part II?',
 'result': "I don't know the answer."}

In [37]:
chain.invoke("Who is the oldest actor who played in The Godfather: Part II?")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (p:Person)-[:ActedIn]->(m:Movie {name: 'The Godfather: Part II'})
RETURN p.name
ORDER BY p.birthDate ASC
LIMIT 1
Full Context:
[{'p.name': 'Al Pacino'}]

> Finished chain.


{'query': 'Who is the oldest actor who played in The Godfather: Part II?',
 'result': 'Al Pacino is the oldest actor who played in The Godfather: Part II.'}

## Use separate LLMs for Cypher and answer generation

You can specify `cypher_llm` and `qa_llm` separately to use different LLMs for Cypher generation and answer generation.

In [ ]:
chain = KuzuQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-4"),
    graph=graph,
    verbose=True,
)

In [38]:
chain.invoke("How many actors played in The Godfather: Part II?")



> Entering new KuzuQAChain chain...
Generated Cypher:
MATCH (:Person)-[:ActedIn]->(:Movie {name: 'The Godfather: Part II'})
RETURN count(*)
Full Context:
[{'COUNT_STAR()': 2}]

> Finished chain.


{'query': 'How many actors played in The Godfather: Part II?',
 'result': 'There were 2 actors who played in The Godfather: Part II.'}